In [5]:
import numpy as np
import pandas as pd

In [6]:
def read_csv(csv_list):
    output = pd.DataFrame()
    for csv in csv_list:
        df = pd.read_csv(csv)
        output = pd.concat([output, df])
    output.reset_index(drop=True, inplace=True)
    return output

def load_database():
    
    # all the ortho filenames
    ortho_df = read_csv(['ortho1200_all.csv'])
    ortho_df['label'] = [False for i in range(len(ortho_df))]
    ortho_df.columns = ortho_df.columns.str.replace('現有檔名', 'NAME')
    ortho_df.NAME = ortho_df.apply(lambda x: x.NAME.split('.')[0], axis=1)
    ortho_df['CloudRate'] = ortho_df['CloudRate'].apply(lambda x: x // 10)
    ortho_df['secret'] = ortho_df.apply(lambda x: x.NAME[-1] == 'c', axis=1)
    
    # all the DMC filenames
    aerial_df = read_csv(['DMC2021_pick_all.csv', 'DMC3_with_quality.csv'])
    aerial_df['label'] = [False for i in range(len(aerial_df))]
    aerial_df['NAME'] = aerial_df.apply(lambda x: x.filename if isinstance(x.filename, str) else x['name'], axis=1)
    aerial_df['secret'] = aerial_df.apply(lambda x: x.NAME[-1] == 'c', axis=1)
    aerial_df.NAME = aerial_df.apply(lambda x: x.NAME.split('.')[0], axis=1)
    return {'ortho': ortho_df[['NAME', 'CloudRate', 'secret', 'label']], 'DMC': aerial_df[['NAME', 'QUALITY', 'secret', 'label']]}

In [13]:
output = load_database()
for phase in ['ortho', 'DMC']:
    output[phase].to_csv(f'{phase}_all.csv', index=False)